In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

TRAIN_CSV = "../data/Training_set.csv"
TEST_CSV  = "../data/Testing_set.csv"
TRAIN_DIR = "../data/train"
TEST_DIR  = "../data/test"

train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

print(train_df.head())
print(test_df.head())
print(train_df['label'].nunique(), "classes")

      filename                     label
0  Image_1.jpg          SOUTHERN DOGFACE
1  Image_2.jpg                    ADONIS
2  Image_3.jpg            BROWN SIPROETA
3  Image_4.jpg                   MONARCH
4  Image_5.jpg  GREEN CELLED CATTLEHEART
      filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg
75 classes


In [11]:
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

train_df = train_df.rename(columns={"Image": "filename", "label": "class"})
test_df  = test_df.rename(columns={"Image": "filename"})

# Train/validation split
from sklearn.model_selection import train_test_split

train_df_split, val_df_split = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df['class'],
    random_state=123
)

train_datagen = ImageDataGenerator(
    rescale=1./255
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    train_df_split,
    directory=TRAIN_DIR,
    x_col="filename",
    y_col="class",
    target_size=IMAGE_SIZE,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df_split,
    directory=TRAIN_DIR,
    x_col="filename",
    y_col="class",
    target_size=IMAGE_SIZE,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False
)

num_classes = train_df_split["class"].nunique()
print("Number of classes:", num_classes)

# Test generator – IMPORTANT: shuffle=False to preserve CSV order
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_dataframe(
    test_df,
    directory=TEST_DIR,
    x_col="filename",
    y_col=None,
    target_size=IMAGE_SIZE,
    class_mode=None,
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 5199 validated image filenames belonging to 75 classes.
Found 1300 validated image filenames belonging to 75 classes.
Number of classes: 75
Found 2786 validated image filenames.


In [12]:
from tensorflow.keras import layers, models

def build_cnn(num_conv_blocks=3, base_filters=32, dense_units=256):
    model = models.Sequential()
    
    # First block (requires input shape)
    model.add(layers.Conv2D(base_filters, (3,3), activation='relu', 
                            input_shape=(*IMAGE_SIZE, 3)))
    model.add(layers.MaxPooling2D((2,2)))

    # Additional blocks
    filters = base_filters
    for i in range(num_conv_blocks - 1):
        filters *= 2
        model.add(layers.Conv2D(filters, (3,3), activation='relu'))
        model.add(layers.MaxPooling2D((2,2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
        
    return model




In [13]:
EPOCHS = 10

depths = [1, 2, 3, 4]   # try what you want
histories = {}

for d in depths:
    print(f"\nTraining model with {d} conv blocks...\n")
    model = build_cnn(num_conv_blocks=d)
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS
    )
    histories[d] = history

    val_loss, val_acc = model.evaluate(val_gen)
    print(f"Validation accuracy for depth {d}: {val_acc:.4f}")



Training model with 1 conv blocks...



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 27s 161ms/step - accuracy: 0.0130 - loss: 6.1498 - val_accuracy: 0.0200 - val_loss: 4.3135
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 206ms/step - accuracy: 0.0165 - loss: 4.3166 - val_accuracy: 0.0185 - val_loss: 4.2953
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 208ms/step - accuracy: 0.0195 - loss: 4.2890 - val_accuracy: 0.0354 - val_loss: 4.1174
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 38s 230ms/step - accuracy: 0.0358 - loss: 4.1491 - val_accuracy: 0.0638 - val_loss: 4.0484
Epoch 5/10
109/163 ━━━━━━━━━━━━━━━━━━━━ 10s 203ms/step - accuracy: 0.0345 - loss: 4.0605

In [ ]:
# Predict class probabilities
pred_probs = model.predict(test_gen)

# Convert to class indices
pred_indices = np.argmax(pred_probs, axis=1)

# Map indices back to labels (strings)
index_to_class = {v: k for k, v in train_gen.class_indices.items()}
pred_labels = [index_to_class[i] for i in pred_indices]

# Attach to test_df in the same order
test_df['predicted_label'] = pred_labels

# Save for the competition / your vignette
test_df.to_csv("../results/butterfly_predictions.csv", index=False)
print(test_df.head())


88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step
      filename         predicted_label
0  Image_1.jpg              PINE WHITE
1  Image_2.jpg           CRIMSON PATCH
2  Image_3.jpg      RED SPOTTED PURPLE
3  Image_4.jpg         IPHICLUS SISTER
4  Image_5.jpg  MILBERTS TORTOISESHELL
